<a href="https://colab.research.google.com/github/TurboPug2101/Sentiment_Analysis/blob/main/Sentiment_Analysis_Revised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Read the data set
data_url = "/content/amazon_reviews.csv"
sentiment_data = pd.read_csv(data_url)



In [ ]:
from datetime import datetime as dt


In [ ]:
sentiment_data

,Unnamed: 0,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
0,0,NaN,4.0,No issues.,2014-07-23,138,0,0,0,0,0.0,0.0
1,1,0mie,5.0,"Purchased this for my device, it worked as adv...",2013-10-25,409,0,0,0,0,0.0,0.0
2,2,1K3,4.0,it works as expected. I should have sprung for...,2012-12-23,715,0,0,0,0,0.0,0.0
3,3,1m2,5.0,This think has worked out great.Had a diff. br...,2013-11-21,382,0,0,0,0,0.0,0.0
4,4,2&amp;1/2Men,5.0,"Bought it with Retail Packaging, arrived legit...",2013-07-13,513,0,0,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4910,4910,"ZM ""J""",1.0,I bought this Sandisk 16GB Class 10 to use wit...,2013-07-23,503,0,0,0,0,0.0,0.0
4911,4911,Zo,5.0,Used this for extending the capabilities of my...,2013-08-22,473,0,0,0,0,0.0,0.0
4912,4912,Z S Liske,5.0,Great card that is very fast and reliable. It ...,2014-03-31,252,0,0,0,0,0.0,0.0
4913,4913,Z Taylor,5.0,Good amount of space for the stuff I want to d...,2013-09-16,448,0,0,0,0,0.0,0.0


In [ ]:
data=sentiment_data[['reviewText','overall']]

In [ ]:
data

,reviewText,overall
0,No issues.,4.0
1,"Purchased this for my device, it worked as adv...",5.0
2,it works as expected. I should have sprung for...,4.0
3,This think has worked out great.Had a diff. br...,5.0
4,"Bought it with Retail Packaging, arrived legit...",5.0
...,...,...
4910,I bought this Sandisk 16GB Class 10 to use wit...,1.0
4911,Used this for extending the capabilities of my...,5.0
4912,Great card that is very fast and reliable. It ...,5.0
4913,Good amount of space for the stuff I want to d...,5.0


Vader

In [ ]:
!pip install nltk
import nltk

# Download the lexicon
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Create an instance of SentimentIntensityAnalyzer
sent_analyzer = SentimentIntensityAnalyzer()


In [ ]:
def format_data(data):

    rev_col = str(data.columns[0])
    sc_col = str(data.columns[-1])

    data.rename(columns = {rev_col: 'Review', sc_col: 'polarity'}, inplace=True)

    # Change 0, 2, 4 to negative, neutral and positive
    labels = {1: 'negative',2: 'negative', 3: 'neutral', 4: 'positive',5: 'positive'}
    data['polarity'] = data['polarity'].map(labels)

    # Get only the two columns
    return data[['Review', 'polarity']]

In [ ]:
dataV=format_data(data)

<ipython-input-7-a812662193df>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns = {rev_col: 'Review', sc_col: 'polarity'}, inplace=True)
<ipython-input-7-a812662193df>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['polarity'] = data['polarity'].map(labels)


In [ ]:
dataV

,Review,polarity
0,No issues.,positive
1,"Purchased this for my device, it worked as adv...",positive
2,it works as expected. I should have sprung for...,positive
3,This think has worked out great.Had a diff. br...,positive
4,"Bought it with Retail Packaging, arrived legit...",positive
...,...,...
4910,I bought this Sandisk 16GB Class 10 to use wit...,negative
4911,Used this for extending the capabilities of my...,positive
4912,Great card that is very fast and reliable. It ...,positive
4913,Good amount of space for the stuff I want to d...,positive


In [ ]:
def format_output(output_dict):

    polarity = "neutral"

    if(output_dict['compound']>= 0.05):
        polarity = "positive"

    elif(output_dict['compound']<= -0.05):
        polarity = "negative"

    return polarity

def predict_sentiment(text):

    output_dict =  sent_analyzer.polarity_scores(text)
    return format_output(output_dict)

In [ ]:
dataV = dataV.dropna(subset=["Review"])


In [ ]:
start = dt.now()
dataV["vader_prediction"] = dataV["Review"].apply(predict_sentiment)
running_secs = (dt.now() - start).seconds


<ipython-input-36-2e8cdadfd193>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataV["vader_prediction"] = dataV["Review"].apply(predict_sentiment)


In [ ]:
dataV

,Review,polarity,vader_prediction
0,No issues.,positive,negative
1,"Purchased this for my device, it worked as adv...",positive,negative
2,it works as expected. I should have sprung for...,positive,negative
3,This think has worked out great.Had a diff. br...,positive,positive
4,"Bought it with Retail Packaging, arrived legit...",positive,positive
...,...,...,...
4910,I bought this Sandisk 16GB Class 10 to use wit...,negative,neutral
4911,Used this for extending the capabilities of my...,positive,positive
4912,Great card that is very fast and reliable. It ...,positive,positive
4913,Good amount of space for the stuff I want to d...,positive,positive


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(dataV['polarity'], dataV['vader_prediction'])

print("Accuracy: {}\n".format(accuracy))

# Show the classification report
print(classification_report(dataV['polarity'], dataV['vader_prediction']))

Accuracy: 0.7568172568172569

              precision    recall  f1-score   support

    negative       0.23      0.59      0.33       324
     neutral       0.04      0.10      0.06       142
    positive       0.95      0.79      0.86      4448

    accuracy                           0.76      4914
   macro avg       0.40      0.49      0.41      4914
weighted avg       0.87      0.76      0.80      4914



In [ ]:
print(running_secs)

2


# Naive Bayes

In [ ]:
import re
def cleaning(text):
    # converting to lowercase, removing URL links, special characters, punctuations...
    # text = text.lower() # converting to lowercase
    text = re.sub('https?://\S+|www\.\S+', '', text) # removing URL links
    text = re.sub(r"\b\d+\b", "", text) # removing number
    text = re.sub('<.*?>+', '', text) # removing special characters,
    text = re.sub('\n', '', text)
    text = re.sub('[’“”…]', '', text)

    # removing short form:

    text=re.sub("isn't",'is not',text)
    text=re.sub("he's",'he is',text)
    text=re.sub("wasn't",'was not',text)
    text=re.sub("there's",'there is',text)
    text=re.sub("couldn't",'could not',text)
    text=re.sub("won't",'will not',text)
    text=re.sub("they're",'they are',text)
    text=re.sub("she's",'she is',text)
    text=re.sub("There's",'there is',text)
    text=re.sub("wouldn't",'would not',text)
    text=re.sub("haven't",'have not',text)
    text=re.sub("That's",'That is',text)
    text=re.sub("you've",'you have',text)
    text=re.sub("He's",'He is',text)
    text=re.sub("what's",'what is',text)
    text=re.sub("weren't",'were not',text)
    text=re.sub("we're",'we are',text)
    text=re.sub("hasn't",'has not',text)
    text=re.sub("you'd",'you would',text)
    text=re.sub("shouldn't",'should not',text)
    text=re.sub("let's",'let us',text)
    text=re.sub("they've",'they have',text)
    text=re.sub("You'll",'You will',text)
    text=re.sub("i'm",'i am',text)
    text=re.sub("we've",'we have',text)
    text=re.sub("it's",'it is',text)
    text=re.sub("don't",'do not',text)
    text=re.sub("that´s",'that is',text)
    text=re.sub("I´m",'I am',text)
    text=re.sub("it’s",'it is',text)
    text=re.sub("she´s",'she is',text)
    text=re.sub("he’s'",'he is',text)
    text=re.sub('I’m','I am',text)
    text=re.sub('I’d','I did',text)
    text=re.sub("he’s'",'he is',text)
    text=re.sub('there’s','there is',text)


    return text



In [ ]:
dataV = dataV.dropna(subset=["Review"])


In [ ]:
dataN = dataV['Review'].apply(cleaning)

In [ ]:
dataN

0                                              No issues.
1       Purchased this for my device, it worked as adv...
2       it works as expected. I should have sprung for...
3       This think has worked out great.Had a diff. br...
4       Bought it with Retail Packaging, arrived legit...
                              ...                        
4910    I bought this Sandisk 16GB Class  to use with ...
4911    Used this for extending the capabilities of my...
4912    Great card that is very fast and reliable. It ...
4913    Good amount of space for the stuff I want to d...
4914    I've heard bad things about this 64gb Micro SD...
Name: Review, Length: 4914, dtype: object

In [ ]:
dataN= pd.DataFrame(dataN)

dataN['Sentiment']=dataV['polarity']

In [ ]:
dataN
# dt=dataN

,Review,Sentiment
0,No issues.,positive
1,"Purchased this for my device, it worked as adv...",positive
2,it works as expected. I should have sprung for...,positive
3,This think has worked out great.Had a diff. br...,positive
4,"Bought it with Retail Packaging, arrived legit...",positive
...,...,...
4910,I bought this Sandisk 16GB Class to use with ...,negative
4911,Used this for extending the capabilities of my...,positive
4912,Great card that is very fast and reliable. It ...,positive
4913,Good amount of space for the stuff I want to d...,positive


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words()

dataN['no_sw'] = dataN['Review'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))


In [ ]:
dataN

,Review,Sentiment,no_sw
0,No issues.,positive,No issues.
1,"Purchased this for my device, it worked as adv...",positive,"Purchased device, worked advertised. You phone..."
2,it works as expected. I should have sprung for...,positive,works expected. I sprung higher capacity. I ma...
3,This think has worked out great.Had a diff. br...,positive,This worked great.Had diff. bran 64gb card sou...
4,"Bought it with Retail Packaging, arrived legit...",positive,"Bought Retail Packaging, arrived legit, orange..."
...,...,...,...
4910,I bought this Sandisk 16GB Class to use with ...,negative,"I bought Sandisk 16GB Class HTC Inspire, month..."
4911,Used this for extending the capabilities of my...,positive,Used extending capabilities Samsung Galaxy Not...
4912,Great card that is very fast and reliable. It ...,positive,Great card fast reliable. It optional adapter ...
4913,Good amount of space for the stuff I want to d...,positive,"Good amount space stuff I do, fits GoPro. What..."


In [ ]:
from collections import Counter
cnt = Counter()
for text in dataN["no_sw"].values:
    for word in text.split():
        cnt[word] += 1
cnt.most_common(10)
temp = pd.DataFrame(cnt.most_common(10))
temp.columns=['word', 'count']
temp

,word,count
0,I,7571
1,card,3318
2,memory,1080
3,works,1078
4,phone,1077
5,SD,889
6,It,883
7,Galaxy,867
8,great,831
9,The,807


In [ ]:
# Remove the most frequent words:
FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])
def remove_freqwords(text):
    """custom function to remove the frequent words"""
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])
dataN["wo_stopfreq"] = dataN["no_sw"].apply(lambda text: remove_freqwords(text))
dataN.head()

,Review,Sentiment,no_sw,wo_stopfreq
0,No issues.,positive,No issues.,No issues.
1,"Purchased this for my device, it worked as adv...",positive,"Purchased device, worked advertised. You phone...","Purchased device, worked advertised. You memor..."
2,it works as expected. I should have sprung for...,positive,works expected. I sprung higher capacity. I ma...,expected. sprung higher capacity. made bit che...
3,This think has worked out great.Had a diff. br...,positive,This worked great.Had diff. bran 64gb card sou...,This worked great.Had diff. bran 64gb south mo...
4,"Bought it with Retail Packaging, arrived legit...",positive,"Bought Retail Packaging, arrived legit, orange...","Bought Retail Packaging, arrived legit, orange..."


In [ ]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
wordnet_lem = WordNetLemmatizer()

dataN['wo_stopfreq_lem'] = dataN['wo_stopfreq'].apply(wordnet_lem.lemmatize)
dataN

[nltk_data] Downloading package wordnet to /root/nltk_data...


,Review,Sentiment,no_sw,wo_stopfreq,wo_stopfreq_lem
0,No issues.,positive,No issues.,No issues.,No issues.
1,"Purchased this for my device, it worked as adv...",positive,"Purchased device, worked advertised. You phone...","Purchased device, worked advertised. You memor...","Purchased device, worked advertised. You memor..."
2,it works as expected. I should have sprung for...,positive,works expected. I sprung higher capacity. I ma...,expected. sprung higher capacity. made bit che...,expected. sprung higher capacity. made bit che...
3,This think has worked out great.Had a diff. br...,positive,This worked great.Had diff. bran 64gb card sou...,This worked great.Had diff. bran 64gb south mo...,This worked great.Had diff. bran 64gb south mo...
4,"Bought it with Retail Packaging, arrived legit...",positive,"Bought Retail Packaging, arrived legit, orange...","Bought Retail Packaging, arrived legit, orange...","Bought Retail Packaging, arrived legit, orange..."
...,...,...,...,...,...
4910,I bought this Sandisk 16GB Class to use with ...,negative,"I bought Sandisk 16GB Class HTC Inspire, month...","bought Sandisk 16GB Class HTC Inspire, months,...","bought Sandisk 16GB Class HTC Inspire, months,..."
4911,Used this for extending the capabilities of my...,positive,Used extending capabilities Samsung Galaxy Not...,Used extending capabilities Samsung Note . gre...,Used extending capabilities Samsung Note . gre...
4912,Great card that is very fast and reliable. It ...,positive,Great card fast reliable. It optional adapter ...,Great fast reliable. optional adapter slots. B...,Great fast reliable. optional adapter slots. B...
4913,Good amount of space for the stuff I want to d...,positive,"Good amount space stuff I do, fits GoPro. What...","Good amount space stuff do, fits GoPro. What say?","Good amount space stuff do, fits GoPro. What say?"


# Tokenization

In [ ]:
nb=dataN.drop(columns=['Review','no_sw', 'wo_stopfreq'])
nb.columns=['sentiment','review']
nb['sentiment'] = nb['sentiment'].map({'positive': 2, 'negative': 0, 'neutral': 1})
nb


,sentiment,review
0,2,No issues.
1,2,"Purchased device, worked advertised. You memor..."
2,2,expected. sprung higher capacity. made bit che...
3,2,This worked great.Had diff. bran 64gb south mo...
4,2,"Bought Retail Packaging, arrived legit, orange..."
...,...,...
4910,0,"bought Sandisk 16GB Class HTC Inspire, months,..."
4911,2,Used extending capabilities Samsung Note . gre...
4912,2,Great fast reliable. optional adapter slots. B...
4913,2,"Good amount space stuff do, fits GoPro. What say?"


In [ ]:
tokenized_review=nb['review'].apply(lambda x: x.split())
tokenized_review.head(5)

0                                        [No, issues.]
1    [Purchased, device,, worked, advertised., You,...
2    [expected., sprung, higher, capacity., made, b...
3    [This, worked, great.Had, diff., bran, 64gb, s...
4    [Bought, Retail, Packaging,, arrived, legit,, ...
Name: review, dtype: object

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)
text_counts = cv.fit_transform(nb['review'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TTS


In [ ]:
from sklearn.model_selection import train_test_split
X=text_counts
y=nb['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,random_state=30)

Complement NB model:


In [ ]:
from sklearn.naive_bayes import ComplementNB
from sklearn.metrics import classification_report, confusion_matrix
CNB = ComplementNB()
CNB.fit(X_train, y_train)

from sklearn import metrics

predicted = CNB.predict(X_test)
accuracy_score = metrics.accuracy_score(predicted, y_test)

print('ComplementNB model accuracy is',str('{:04.2f}'.format(accuracy_score*100))+'%')
print('------------------------------------------------')

print('Classification Report:')
print(classification_report(y_test, predicted))

ComplementNB model accuracy is 88.50%
------------------------------------------------
Classification Report:
              precision    recall  f1-score   support

           0       0.48      0.72      0.58        75
           1       0.05      0.04      0.04        27
           2       0.96      0.93      0.94       881

    accuracy                           0.89       983
   macro avg       0.50      0.56      0.52       983
weighted avg       0.90      0.89      0.89       983



In [ ]:
import time
from sklearn.naive_bayes import ComplementNB
from sklearn.metrics import classification_report, confusion_matrix
import sklearn.metrics as metrics
start_time = time.time()
CNB = ComplementNB()
CNB.fit(X_train, y_train)
training_time = time.time() - start_time

# Make predictions, measuring the prediction time
start_time = time.time()
predicted = CNB.predict(X_test)
prediction_time = time.time() - start_time

# Calculate accuracy
accuracy_score = metrics.accuracy_score(predicted, y_test)

print('ComplementNB model accuracy is', str('{:04.2f}'.format(accuracy_score * 100)) + '%')
print('Training time:', '{:04.2f}'.format(training_time), 'seconds')
print('Prediction time:', '{:04.2f}'.format(prediction_time), 'seconds')
print('------------------------------------------------')

print('Classification Report:')
print(classification_report(y_test, predicted))

ComplementNB model accuracy is 88.50%
Training time: 0.01 seconds
Prediction time: 0.00 seconds
------------------------------------------------
Classification Report:
              precision    recall  f1-score   support

           0       0.48      0.72      0.58        75
           1       0.05      0.04      0.04        27
           2       0.96      0.93      0.94       881

    accuracy                           0.89       983
   macro avg       0.50      0.56      0.52       983
weighted avg       0.90      0.89      0.89       983



# RNN

In [ ]:
import nltk # Natural Language Toolkit
nltk.download('stopwords')
from nltk.corpus import stopwords
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from numpy import array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding, LSTM, Bidirectional
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
dataN

,Review,Sentiment,no_sw,wo_stopfreq,wo_stopfreq_lem
0,No issues.,positive,No issues.,No issues.,No issues.
1,"Purchased this for my device, it worked as adv...",positive,"Purchased device, worked advertised. You phone...","Purchased device, worked advertised. You memor...","Purchased device, worked advertised. You memor..."
2,it works as expected. I should have sprung for...,positive,works expected. I sprung higher capacity. I ma...,expected. sprung higher capacity. made bit che...,expected. sprung higher capacity. made bit che...
3,This think has worked out great.Had a diff. br...,positive,This worked great.Had diff. bran 64gb card sou...,This worked great.Had diff. bran 64gb south mo...,This worked great.Had diff. bran 64gb south mo...
4,"Bought it with Retail Packaging, arrived legit...",positive,"Bought Retail Packaging, arrived legit, orange...","Bought Retail Packaging, arrived legit, orange...","Bought Retail Packaging, arrived legit, orange..."
...,...,...,...,...,...
4910,I bought this Sandisk 16GB Class to use with ...,negative,"I bought Sandisk 16GB Class HTC Inspire, month...","bought Sandisk 16GB Class HTC Inspire, months,...","bought Sandisk 16GB Class HTC Inspire, months,..."
4911,Used this for extending the capabilities of my...,positive,Used extending capabilities Samsung Galaxy Not...,Used extending capabilities Samsung Note . gre...,Used extending capabilities Samsung Note . gre...
4912,Great card that is very fast and reliable. It ...,positive,Great card fast reliable. It optional adapter ...,Great fast reliable. optional adapter slots. B...,Great fast reliable. optional adapter slots. B...
4913,Good amount of space for the stuff I want to d...,positive,"Good amount space stuff I do, fits GoPro. What...","Good amount space stuff do, fits GoPro. What say?","Good amount space stuff do, fits GoPro. What say?"


In [ ]:
labelencoder = LabelEncoder()
dataN = dataN.assign(Sentiment = labelencoder.fit_transform(dataN["Sentiment"]))

In [ ]:
docs = dataN['Review']
labels = array(dataN['Sentiment'])

In [ ]:
labels

array([2, 2, 2, ..., 2, 2, 2])

In [ ]:
X_train, X_test , y_train, y_test = train_test_split(docs, labels , test_size = 0.20)

In [ ]:
X_train

1765    Works well for the Galaxy S3 and provides addi...
2651    Yes it meet me expectation. This is the best h...
3896    A sweet spot for MicroSDHC's.  Added a nice bu...
4891    This was the fastest microSD card I have teste...
4678    Not cheap but really quite amazing that they c...
                              ...                        
1769    Card is well constructed and delivers the wrig...
3425    I bought this 64GB memory card for my mini-cam...
4134    So far have not got any issues. I'm using it t...
3573    Screaming deal on this retail-packaged 32gb mi...
3062    That card was in very special offer, and I kep...
Name: Review, Length: 3931, dtype: object

In [ ]:
max_words = 1000  # Maximum number of words to consider
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)

# Pad sequences so that they have the same length
max_sequence_length = 100  # Adjust this based on your data
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length)


In [ ]:
from keras.layers import Embedding, SimpleRNN, Dense

model = Sequential()

model.add(Embedding(input_dim=max_words, output_dim=32, input_length=max_sequence_length))
model.add(SimpleRNN(units=32, activation='relu'))
model.add(Dense(3, activation='softmax'))  # Assuming there are 3 classes

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
start_time = time.time()
model.fit(X_train_padded, y_train, epochs=5, batch_size=64, validation_split=0.2)
end_time = time.time()


Epoch 1/5
50/50 [==============================] - 13s 225ms/step - loss: 0.4300 - accuracy: 0.8989 - val_loss: 0.2874 - val_accuracy: 0.9238
Epoch 2/5
50/50 [==============================] - 8s 170ms/step - loss: 0.3336 - accuracy: 0.9033 - val_loss: 0.2629 - val_accuracy: 0.9238
Epoch 3/5
50/50 [==============================] - 6s 131ms/step - loss: 0.2847 - accuracy: 0.9036 - val_loss: 0.2395 - val_accuracy: 0.9238
Epoch 4/5
50/50 [==============================] - 7s 142ms/step - loss: 0.2392 - accuracy: 0.9059 - val_loss: 0.2380 - val_accuracy: 0.9238
Epoch 5/5
50/50 [==============================] - 5s 95ms/step - loss: 0.2204 - accuracy: 0.9125 - val_loss: 0.2302 - val_accuracy: 0.9263


In [ ]:
from keras.layers import Embedding, SimpleRNN, Dense

model = Sequential()

model.add(Embedding(input_dim=max_words, output_dim=32, input_length=max_sequence_length))
model.add(SimpleRNN(units=32, activation='relu'))
model.add(Dense(3, activation='softmax'))  # Assuming there are 3 classes

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
start_time = time.time()
model.fit(X_train_padded, y_train, epochs=5, batch_size=64, validation_split=0.2)
end_time = time.time()


Epoch 1/5
50/50 [==============================] - 11s 186ms/step - loss: 0.7230 - accuracy: 0.8785 - val_loss: 0.3585 - val_accuracy: 0.9238
Epoch 2/5
50/50 [==============================] - 7s 151ms/step - loss: 0.3692 - accuracy: 0.9033 - val_loss: 0.2910 - val_accuracy: 0.9238
Epoch 3/5
50/50 [==============================] - 6s 118ms/step - loss: 0.2965 - accuracy: 0.9033 - val_loss: 0.2488 - val_accuracy: 0.9238
Epoch 4/5
50/50 [==============================] - 6s 129ms/step - loss: 0.2493 - accuracy: 0.9033 - val_loss: 0.2606 - val_accuracy: 0.9238
Epoch 5/5
50/50 [==============================] - 5s 91ms/step - loss: 0.2167 - accuracy: 0.9062 - val_loss: 0.2387 - val_accuracy: 0.9263


In [ ]:
X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length)


In [ ]:
# Predict using the trained model
y_pred = model.predict(X_test_padded)
# Convert predicted probabilities to class labels
y_pred_labels = np.argmax(y_pred, axis=1)
correct_predictions = np.sum(y_pred_labels == y_test)
total_samples = len(y_test)
accuracy = correct_predictions / total_samples
print(f"Test Accuracy: {accuracy:.2f}")

31/31 [==============================] - 0s 10ms/step
Test Accuracy: 0.91


In [ ]:
class_names = ["Negative", "Neutral", "Positive"]  # Update with your actual class names
report = classification_report(y_test, y_pred_labels, target_names=class_names)
print("Classification Report:\n", report)

Classification Report:
               precision    recall  f1-score   support

    Negative       0.85      0.15      0.26        72
     Neutral       0.00      0.00      0.00        30
    Positive       0.91      1.00      0.95       881

    accuracy                           0.91       983
   macro avg       0.58      0.38      0.40       983
weighted avg       0.88      0.91      0.87       983



In [ ]:
prediction_time = end_time - start_time
\

print(f"Prediction Time: {prediction_time:.2f} seconds")

Prediction Time: 41.94 seconds
